Example of using Motion Artifact Correction Training Pipeline Using `wat_stacked_unets` Model with TensorBoard Visualization

---

**Description:**  
This notebook demonstrates a complete training pipeline for a motion artifact correction model using a custom architecture called `wat_stacked_unets`. It covers:  

- **Environment setup:** Installing TensorFlow and required libraries.  
- **Configuration setup:** Preparing a configuration file to define dataset paths, model settings, hyperparameters, and training callbacks.  
- **Codebase integration:** Cloning the necessary codebase from GitHub.  
- **Training execution:** Initiating training with the specified configuration.  
- **Visualization:** Setting up TensorBoard with Ngrok to monitor training metrics remotely.  

This setup is tailored for experimenting with multiple datasets, comparing loss functions, and visualizing training progress with TensorBoard.

## 1. Environment Setup

In [ ]:
# Install TensorFlow
!pip install -U tensorflow==2.15.0

# Import necessary libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import os

# Print TensorFlow version
print("TensorFlow version:", tf.__version__)
# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

TensorFlow version: 2.15.0


## 2. Configuration Setup

In [ ]:
import json

# Configuration data for training
config_data = {
    "dataset": "/kaggle/input/mmmai-regist-data/MR-ART-Regist",
    "data_ids": {
        "/kaggle/input/mmmai-simulated-data/ds004795-download": "Motion_Simulated",
        "/kaggle/input/mmmai-regist-data/MR-ART-Regist": "Motion",
        "/kaggle/input/brats-motion-data/new_Brats_motion_data": "BraTS"
    },
    "loss_functions": [
        "ssim_loss",
        "perceptual_loss",
    ],
    "model_architecture": "wat_stacked_unets",
    "checkpoint_path": None,
    "epochs": 3,
    "batch_size": 10,
    "enable_SAP": True,
    "start_epoch": 0,
    "learning_rate": 0.001,
    "height": 256,
    "width": 256,
    "token": "cr_2amWxkJ1fUvF7hSBd1FRrBNk6ii",
    "data_loader": {
        "split_ratio": [0.7, 0.2, 0.1],
        "view": "Axial",
        "crop": False,
        "split_json_path": None
    },
    "weights_path": "/kaggle/working/",
    "callbacks": {
        "csv_logger": {
            "enabled": True,
            "filename": "_Loss_Acc.csv"
        },
        "reduce_lr": {
            "enabled": True,
            "schedule_function": "exponential_lr"
        },
        "model_checkpoint": {
            "enabled": True,
            "filename_pattern": "wat_stacked_unets_epoch-{epoch:02d}_val_loss-{val_loss:.4f}.h5"
        }
    }
}

# Save configuration to a JSON file
json_file_path = "/kaggle/working/config.json"
with open(json_file_path, "w") as json_file:
    json.dump(config_data, json_file, indent=4)

print(f"Configuration file saved at: {json_file_path}")

JSON file saved at: /kaggle/working/config.json


## 3. Clone the Codebase

In [ ]:
# Clone the GitHub repository containing the training code
!git clone https://github.com/mahmoud1yaser/mmmai_codebase

# Change the working directory
%cd /kaggle/working/mmmai_codebase

## 4. Start the Training Process

In [ ]:
# Run the training script using the generated configuration file
!python /kaggle/working/mmmai_codebase/src/training/trainer.py --config /kaggle/working/config.json

## 5. Set Up Ngrok for TensorBoard

In [ ]:
# Download and extract Ngrok
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar xf ./ngrok-v3-stable-linux-amd64.tgz -C /usr/local/bin

# Authorize Ngrok with the token from the configuration file
!ngrok authtoken {config_data['token']}

## 6. Launch TensorBoard and Ngrok

In [ ]:
import multiprocessing

# Launch TensorBoard and Ngrok using multiprocessing
def run_command(cmd):
    os.system(cmd)

commands = [
    "tensorboard --logdir /kaggle/working/logs --load_fast=false --host 0.0.0.0 --port 6006 &",
    "/usr/local/bin/ngrok http 6006 &"
]

with multiprocessing.Pool(processes=len(commands)) as pool:
    pool.map(run_command, commands)

## 7. Retrieve Ngrok URL

In [ ]:
# Fetch the public URL for TensorBoard from Ngrok
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"